### 7-1　横持ちへの変換

In [1]:
import pandas as pd

In [2]:
reserve_tb = pd.read_csv('reserve.csv')

print(reserve_tb.shape)
reserve_tb.head()

(4030, 9)


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


customer_idとpeople_numにおいて、縦に値の重複をしている。

In [3]:
pd.pivot_table(reserve_tb, index='customer_id', columns='people_num', values='reserve_id',
               aggfunc=lambda x: len(x), fill_value=0).head()

# 同じcustomer1でも、予約回数は5回以上である、それをpeople_numごとにカウントしている

people_num,1,2,3,4
customer_id,,,,
c_1,2,2,2,2
c_10,0,2,2,2
c_100,2,1,2,0
c_1000,1,0,0,1
c_101,2,1,1,1


### 7-2　スパースマトリックス

内部で縦持ちのデータ表現のまま保持しながら、インターフェースでは行列（表）を持つように表現する。

In [4]:
from scipy.sparse import csc_matrix

cnt_tb = reserve_tb.groupby(['customer_id', 'people_num'])['reserve_id'].size()
cnt_tb.head()

# インデックスが、customer_idそのものになってしまっている

customer_id  people_num
c_1          1             2
             2             2
             3             2
             4             2
c_10         2             2
Name: reserve_id, dtype: int64

In [5]:
# インデックスに番号を振り直す、かつデータフレーム化する
cnt_tb = reserve_tb.groupby(['customer_id', 'people_num'])['reserve_id'].size().reset_index()
cnt_tb.head()

,customer_id,people_num,reserve_id
0,c_1,1,2
1,c_1,2,2
2,c_1,3,2
3,c_1,4,2
4,c_10,2,2


In [6]:
cnt_tb.columns = ['customer_id', 'people_num', 'rsv_cnt']

print(cnt_tb.dtypes)
cnt_tb.head()

customer_id    object
people_num      int64
rsv_cnt         int64
dtype: object


,customer_id,people_num,rsv_cnt
0,c_1,1,2
1,c_1,2,2
2,c_1,3,2
3,c_1,4,2
4,c_10,2,2


In [7]:
# 列の値をカテゴリ型に変換
customer_id = pd.Categorical(cnt_tb['customer_id'])
people_num = pd.Categorical(cnt_tb['people_num'])

print(type(customer_id))
print(type(people_num))

<class 'pandas.core.arrays.categorical.Categorical'>
<class 'pandas.core.arrays.categorical.Categorical'>


In [8]:
csc_matrix((
    
    # 横持ちデータ（値、行番号、列番号）
    cnt_tb['rsv_cnt'], (customer_id.codes, people_num.codes)),
    
    # スパースマトリックスのサイズ（行／列）を指定
    shape=(len(customer_id.categories), len(people_num.categories)))

<888x4 sparse matrix of type '<class 'numpy.int64'>'
	with 2366 stored elements in Compressed Sparse Column format>

In [9]:
# インデックス番号の取得
print(customer_id.codes)

# 列番号の取得
print(people_num.codes)

[  0   0   0 ... 887 887 887]
[0 1 2 ... 1 2 3]


In [10]:
# customer_idのユニークな数
print(len(customer_id.categories))

# people_numのユニークな数
print(len(people_num.categories))

888
4
